#### 一連の手続き.ipynbでやってきた処理をひとまとめにする。考慮すべき事項は下につけ足していくことにする。

In [19]:
#必要なモジュールのimport 
import astropy.io.fits as fits
import numpy as np
import glob
import os
from scipy.ndimage import gaussian_filter
import pandas as pd
import shutil


def readheader(filename):
    return fits.open(filename)[0].header


def readdata(filename):
    return fits.open(filename)[0].data


ほかのファイルでダークのコマンドリストを作成する。
そのコマンドから読み取っていくことにする。

In [20]:
#天体画像を一つ指定したとする　→　obsfile
#darkのcsvファイルから、天体の画像の、日付、PIXTIM, RRSTRT, YSTRTが等しいものを取得する
#darklist = pd.read_csv(filename,header, skiprows)

#def judge(information):
#   darklist[information] == obsfileheader[information]
#judge("DATE-OBS") & judge("Q_PIXTIM") & judge("Q_RRSTRT") & judge("YSTRT")
# filepath = os.path.join()

In [21]:
obsfilename = #入力
dark1 = #入力
dark2 = #入力
obs_comq = #入力

SyntaxError: invalid syntax (4072968027.py, line 1)

### darkが欲しいものと一致しているとして処理をする

In [22]:
#関数の定義
def readheader(filename):
    return fits.open(filename)[0].header

def readdata(filename):
    return fits.open(filename)[0].data

def q_bsep_posi(input):
    return input[:,::2,:,:]
def q_bsep_nega(input):
    return input[:,1::2,:,:]   
#z方向平均
def mean_z(input):
    return np.mean(input, axis=1, keepdims=True)

#gaussian filter


def gaussfilter(input):
    output = gaussian_filter(input[0, 0, :, :], sigma=6, order=0,
                             output=None, mode='nearest', cval=0.0, truncate=4.0)
    return output


def q_subch(input):
    #z方向については、COMQは1となっているはずである。
    input_dim2 = input[0, 0, :, :]
    #x軸方向に20*240と分割し、z方向に積み上げる

    divstack = np.stack(np.split(input_dim2, 16, axis=1))
    #z方向にmedianをとり、20*240をtileして320*240にする
    beforetile = np.median(divstack, axis=0)  # keepdimsは不要(二次元でいい)
    tilemedian = np.tile(beforetile, 16)
    # print(tilemedian.shape)
    input_to_output = input.copy()
    input_to_output[0, 0, :, :] -= tilemedian
    # input_to_output[0, 0, :, :] = 1
    return input_to_output

#fitsファイル作成用

def makefits(dataname, filename):
    hdu = fits.PrimaryHDU(data=dataname)
    fits.HDUList([hdu]).writeto(str(filename), overwrite=True)


In [4]:
# darkheader1 = readheader(dark1)
# darkheader2 = readheader(dark2)
# obsheader = readheader(obsfilename)
# print(darkheader1["Q_CHEB"] == darkheader2["Q_CHEB"])
# print()
def process(obsfilename):
    
    """  
    ここには天体画像から、ダークを特定する処理を記載する。
    
    """
    darkheader1  = 
    if (darkheader1["Q_CHAM"] != 1) or (obsheader["Q_CHAM"]!=1):
        print("CoAdd Modeが1ではない。")
    elif darkheader1["Q_CHAM"]==1 and darkheader2["Q_CHAM"]==1:
        darkfile1 = readdata(dark1)
        darkfile2 = readdata(dark2)
        meandark = (np.mean(dark1, axis=1, keepdims=True)+np.mean(dark2, axis=1, keepdims=True)) /2
        meandark_1 = meandark / int(darkheader1["Q_CHEB"])

        #darkの差し引き
        #Q_CHEB倍する
        obsdata = readdata(obsfilename)
        meandark_obs = meandark_1 * obsheader["Q_CHEB"]
        
        skydata = obsdata - meandark_obs
        skydata_p = q_bsep_posi(skydata)
        skydata_n = q_bsep_nega(skydata)
        
        #z方向平均
        sky_pa = mean_z(skydata_p)
        sky_na = mean_z(skydata_n)
        
        #gaussian　平均
        sky_paG = gaussfilter(sky_pa)
        sky_naG = gaussfilter(sky_na)
        
        #Flatの作成
        sky_paF = sky_pa / sky_paG
        sky_naF = sky_na / sky_naG
        
        #Chop差し引きした、COMQ画像に対して、Flatで割る
        comq_data = readdata(obs_comq)
        obj_obs = q_subch(comq_data)
        
        #Flatで割る
        obj_obs_datP0 = obj_obs / sky_naF
        obj_obs_datN0 = obj_obs / sky_paF    
        
        makefits(obj_obs_datP0, "./out/obj"+
                str(obsfilename[-10:-5]) + "_datP0.fits")
        makefits(obj_obs_datN0, "./out/obj" +
                str(obsfilename[-10:-5]) + "_datN0.fits")


### processの改良版、フォルダ整理から順番を書いていく
流れ  
⓪フォルダは、DATE - > OBJ | DARKとする。  
①天体画像を選択(for文で)  
②天体画像から、Q_PIXTIM, Q_RRSTRT, DATE, Q_YSTRTの一致するダーク画像を選ぶ  
ダークを選ぶ際、天体画像のDATEからQ_CHEBが一致するものに絞る  
③複数枚のダークから、z方向平均をとり、ダークの枚数で平均をとる。    
④解析処理  
⑤outputフォルダに入れること。  
この際、ヘッダーはinput画像と同一のものにする。

#### 2フォルダの整理(練習用)
ダウンロードしてきたファイルは、ダークと天体画像が混ざっている。
そこで、いったん、ダークだけを取り出し別のフォルダに移すことを考える。
FRAME ID のリストからDATA_TYPあるいは、OBJECT2がDARKであればダークと判断する。
名前が一致するものをファイルの中から探し、見つかればファイルをダークフォルダに移動する。

2009-01-13のデータで練習してみる。

'D:\2009-01-13\P03yushi1102114609NN'

In [5]:
folderpath = r'D:\2009-01-13'
judgedarkfile = r'./DataFrame/COM_2009.xlsx'


In [6]:
frameall = pd.read_excel(judgedarkfile)
#1行目は不要なので削除
frameall = frameall.dropna(how="all",axis=0)
darkframe= frameall[(frameall["DATA_TYP"] =="DARK") | (frameall["OBJECT2"]=="DARK")]
#indexをリセット
darkframe = darkframe.reset_index(drop=True)

In [7]:
#darkframeのうち、日付で1/13を選択
darkframe_09_0113 = darkframe[darkframe["DATE_OBS"]=="2009-01-13"]
# darkframe_09_0113

ここまででダークは取ってこられたので、該当するものをダークフォルダを作成し移動していく

In [8]:
#ダークフォルダを作成
try:
    os.mkdir(r"D:\2009-01-13\DARK")
except FileExistsError:
    pass

#あるいは
os.makedirs(r"D:\2009-01-13\DARK",exist_ok=True)

#filepath
filepath = r"D:\2009-01-13"
filelist = glob.glob(filepath+"\*")

#darkの名前を取得
for dark in darkframe_09_0113["#FRAME_ID"]:
    #filename作成
    name = filepath +"\\"+ str(dark) +".fits"
    #移動
    if name in filelist:
        shutil.move(name, filepath+"\\DARK\\" + str(dark) +".fits")

In [9]:
darkfilepath = r"D:\2009-01-13\DARK"
darkfilelist = glob.glob(darkfilepath+"\\*")
d1 = darkfilelist[0]

In [10]:
d1head = readheader(d1)
d1head["FRAMEID"]
d1head["DATE-OBS"]
d1head["Q_PIXTIM"]
d1head["Q_RRSTRT"]
d1head["Q_YSTRT"]
d1head["Q_CHAM"]

1

In [15]:
#天体画像特定のためのDARKの情報を入れたデータフレームを作成する。
darkdf = pd.DataFrame(columns=["FRRAME_ID", "DATE", "PIXTIM", "RRSTRT", "YSTRT", "CoAdd"])
for darkfile in darkfilelist:
    darkheader = readheader(darkfile)
    # darkdf.iloc[len(darkdf)] = [darkheader["FRAMEID"], darkheader["DATE-OBS"], darkheader["Q_PIXTIM"], darkheader["Q_RRSTRT"], darkheader["Q_YSTRT"], darkheader["Q_CHAM"]]
    # [darkheader["FRAMEID"], darkheader["DATE-OBS"], darkheader["Q_PIXTIM"],
    #     darkheader["Q_RRSTRT"], darkheader["Q_YSTRT"], darkheader["Q_CHAM"]]
    list1 = [darkheader["FRAMEID"], darkheader["DATE-OBS"], darkheader["Q_PIXTIM"],
            darkheader["Q_RRSTRT"], darkheader["Q_YSTRT"], darkheader["Q_CHAM"]]
    add_row = pd.DataFrame([list1], columns = darkdf.columns)
    darkdf = darkdf.append(add_row, ignore_index=True)
darkdf
#pd.DataFrame(list)は列として認識される
#pd.DataFrame([list])とすれば解決するっぽい

C:\Users\yyush\AppData\Local\Temp\ipykernel_42604\1064128850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  darkdf = darkdf.append(add_row, ignore_index=True)


,FRRAME_ID,DATE,PIXTIM,RRSTRT,YSTRT,CoAdd
0,COMA00097351,2009-01-13,30,200,1,1
1,COMA00097353,2009-01-13,30,200,1,1
2,COMA00097355,2009-01-13,30,160,1,1
3,COMA00097357,2009-01-13,30,160,1,1
4,COMA00097359,2009-01-13,30,1,80,1
5,COMA00097361,2009-01-13,30,1,80,1
6,COMA00097363,2009-01-13,30,120,1,1
7,COMA00097365,2009-01-13,30,120,1,1
8,COMA00097367,2009-01-13,30,1,60,1
9,COMA00097369,2009-01-13,30,1,60,1


### 3実際のデータのフォルダ整理

D:\P03*
の中にあるファイルをすべて読み取る。


In [54]:
folderpath = r'D:\P03yushi*\*.fits'
filelist = glob.glob(folderpath)
len(filelist)

625

ファイルからheaderを読み取る
その後、headerの中で、objectnameと日付、あるいはobjecttypeを見て、フォルダを分類する。

In [55]:
filelist[500]
header = readheader(filelist[500])
header

SIMPLE  =                    T / Standard FITS format                           
BITPIX  =                   32 / # of bits per pixel                            
NAXIS   =                    4 / # of axis in data                              
NAXIS1  =                  320 / # of pixels/row                                
NAXIS2  =                  240 / # of rows                                      
NAXIS3  =                    1 / # of frames                                    
NAXIS4  =                    1 / # of detectors                                 
EXTEND  =                    F / ASCII Extension Table                          
COMMENT = ' +++++++++++++++++++ SUBARU COMMON'                                  
COMMENT = ' ------------------------- About This DATA'                          
FRAMEID = 'COMA00097219'       / Data Serial Num of COMICS A-sequence           
EXP-ID  = 'COME00097219'       / Exporsure ID                                   
OBS-ALOC= 'Observation '    

## フォルダー移動完成版

In [65]:
folderpath = r'D:\P03yushi*\*.fits'
filelist = glob.glob(folderpath)

for file in filelist:
    header = readheader(file)
    objectname = header["OBJECT"]
    dateobs = header["DATE-OBS"]
    newpath = r'D:\\' + dateobs + r'\\'+objectname
    os.makedirs(newpath,exist_ok=True)
    try:
        shutil.move(file, newpath+r'\\')
    except:
        pass

#フォルダの作成や移動は完了
#空になっていれば削除

afterfilelist = glob.glob(folderpath)
if len(afterfilelist) == 0:
    print(len(afterfilelist))
    
    shutil.rmtree('D:\\P03yushi*\\')

0


OSError: [WinError 123] ファイル名、ディレクトリ名、またはボリューム ラベルの構文が間違っています。: 'D:\\P03yushi*\\'